In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread

%matplotlib inline

In [ ]:
prefix = '/n/projects/'

checkpoint = prefix + 'cjw/Data/sez/Checkpoints/2019-03-11-t/dnet_cat_checkpoint-2500'
#checkpoint = '/ssd1/cjw/Data/sez/Checkpoints/2019-03-07-ft/dnet_cat_checkpoint-2500'
cpmeta = checkpoint + '.meta'

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.import_meta_graph(cpmeta)
saver.restore(sess, checkpoint)

In [ ]:
g = tf.get_default_graph()
ops = g.get_operations()
for op in ops:
    try:
        print(op.name, op.outputs, len(op.outputs))
    except:
        print(op.name, "no output")
    print('------')

In [ ]:
decoder = sess.graph.get_tensor_by_name('decoder-softmax:0')
images = sess.graph.get_tensor_by_name('Placeholder:0')

In [ ]:
import os
xdir = prefix + 'cjw/Data/sez/pxk/'
files = np.array(sorted(os.listdir(xdir)))
files = files.reshape((-1, 3))


In [ ]:
def get_image(filenames, xpath):
    chans = list()
    for f in filenames:
        pf = xpath + f
        data_raw = imread(pf)
        #data_raw = data_raw[:,:,:]
        dmin = data_raw.min(axis=(0,1),keepdims=True)
        dmax = data_raw.max(axis=(0,1),keepdims=True)
        #data = (data_raw - dmin)/(dmax - dmin)
        ymean = data_raw.mean(axis=(0,1), keepdims=True)
        ystd = data_raw.std(axis=(0,1), keepdims=True)
        #test_tif = (test_tif - ymin)/(ymax - ymin + 0.00001)
        chans.append((data_raw - ymean)/ystd)

    return np.stack(chans, axis=-1)

# xd = get_image(files[0], xdir)
# print(xd.shape)
# plt.figure(figsize=(16,16))
# plt.imshow(xd[:,:,:], cmap='gray')
# print(xd.shape)

In [ ]:
def decode(patches):
    ds = sess.run(decoder, feed_dict={images:patches})
    return ds

        
def image_to_patches(image, w):
    
    ny, nx, nc = image.shape
    
    patch_list = list()
    xmax = 0
    ymax = 0
    xok = True
    yok = True
    while xok:
        yok = True
        ymax = 0
        xs = xmax
        xmax += w
        if xmax >= nx:
            xmax = nx
            xs = nx - w
            xok = False
        while yok:
            ys = ymax
            ymax += w
            if ymax > ny:
                ymax = ny
                ys = ny - w
                yok = False
            #print(ys, ymax, yok, xs, xmax, xok)
            p = np.expand_dims(image[ys:ymax, xs:xmax, :], 0)
            patch_list.append(p)
    
    patches = np.concatenate(patch_list, axis=0)
    #print(patches.shape)
    return  patches

def patches_to_image(patches, w, nx, ny):
    
    image = np.zeros((ny, nx, patches.shape[-1]), dtype=patches.dtype)
    xmax = 0
    ymax = 0
    xok = True
    yok = True
    patch_index = 0
    while xok:
        ymax = 0
        yok = True
        xs = xmax
        xmax += w
        if xmax >= nx:
            xmax = nx
            xs = nx - w
            xok = False
        while yok:
            ys = ymax
            ymax += w
            if ymax > ny:
                ymax = ny
                ys = ny - w
                yok = False
            
            image[ys:ymax, xs:xmax, :] = patches[patch_index]
            patch_index += 1
            #print(patch_index, ys, ymax, xs, xmax, yok, xok)
    return image

In [ ]:
import time
import tifffile

tiffdir = prefix + 'cjw/Data/sez/pxk_inferred_2019-03-18_00/'
if not os.path.exists(tiffdir):
    os.mkdir(tiffdir)

for i, ft in enumerate(files):
#     if i != 79:
#         continue
    t1 = time.time()
    data = get_image(ft, xdir)
    data = np.expand_dims(data[:,:,-1], -1)
    p = image_to_patches(data, 256)
    d2 = data[128:, 128:, :]
    p2 = image_to_patches(d2, 256)
    dp = decode(p)
    dp2 = decode(p2)
    st1 = "Decode Time for " +  ft[0] + ': ' +  "{}".format(time.time()  - t1)
    
    t2 = time.time()
    
    #dp[:,0:3,:,:] = 0
    #dp[:,:,0:3,:] = 0
    dp[:,-2:,:,:] = 0
    dp[:,:,-2:,:] = 0

    dp2[:,0:2,:,:] = 0
    dp2[:,:,0:2,:] = 0
    #dp2[:,-3:,:,:] = 0
    #dp2[:,:,-3:,:] = 0
    
    r1 = patches_to_image(dp, 256, data.shape[1] , data.shape[0])
    r2 = patches_to_image(dp2, 256, data.shape[1] - 128, data.shape[0] - 128)

    rr2 = r1*0
    rr2[128:, 128:, :] = r2
    rstack = np.stack((r1, rr2), axis=0)
    r = rstack.max(axis=0)
    
    tfs = tiffdir + 'pxk_' + str(i + 1).zfill(3) + '.tif'
    #print(tfs)
    tifffile.imsave(tfs, np.moveaxis(r, -1, 0))
    print(st1, " -- Recon Time: ", time.time() - t2)
    #break
    
# plt.subplot(1,3,1)
# plt.imshow(dp[200])
# plt.subplot(1,3,2)
# plt.imshow(dp[201])
# plt.subplot(1,3,3)
# plt.imshow(dp2[200])

In [ ]:
t1 = time.time()

# dp[:,0:2,:,:] = 0
# dp[:,:,0:2,:] = 0
# dp[:,-2:-1,:,:] = 0
# dp[:,:,-2:-1,:] = 0

# dp2[:,0:2,:,:] = 0
# dp2[:,:,0:2,:] = 0
# dp2[:,-2:-1,:,:] = 0
# dp2[:,:,-2:-1,:] = 0

r1 = patches_to_image(dp, 256, data.shape[1] , data.shape[0])
r2 = patches_to_image(dp2, 256, data.shape[1] - 128, data.shape[0] - 128)

rr2 = r1*0
rr2[128:, 128:, :] = r2

rstack = np.stack((r1, rr2), axis=0)

r = rstack.max(axis=0)

print(time.time()  - t1)

In [ ]:
r.shape

In [ ]:
#plt.cla()
tr = r > .5
#tr = tr.astype(np.int32)
#tr = 255*tr
print(tr.shape)
plt.figure(figsize=(8, 8))

plt.imshow(r[1000:1512,1000:1512,:])
plt.show()

In [ ]:
xg = 900 + 128
yg = 700

dr = np.expand_dims(data[yg:yg + 256, xg:xg+256,:], 0)
# dr = np.expand_dims(dr, -1)
da = decode(dr)
print(da.shape)
plt.imshow(((0*da[0,:,:,0] + da[0,:,:,1])/1))